In [ ]:
!pip install looker_sdk

import looker_sdk
import os 
import json 

In [ ]:
#@title Configuration

model_name = 'firebase' #@param {type:"string"}

instance_url = 'https://YOURS.looker.com:19999' #@param {type:"string"}

#Get the following values from your Users page in the Admin panel of your Looker instance > Users > Your user > Edit API keys. If you know your user id, you can visit https://your.looker.com/admin/users/<your_user_id>/edit.
client_id =  'yours' #@param {type:"string"}
client_secret = 'yours' #@param {type:"string"}

print("All environment variables set.")
os.environ["LOOKERSDK_BASE_URL"] = instance_url
os.environ["LOOKERSDK_CLIENT_ID"] = client_id
os.environ["LOOKERSDK_CLIENT_SECRET"] = client_secret
print(os.environ["LOOKERSDK_BASE_URL"])

In [ ]:
sdk = looker_sdk.init31()
my_user = sdk.me()

print("Hi, " + my_user.first_name)



query = looker_sdk.models40.WriteQuery(
   fields=['events.event_name','events__event_params.key','events__event_params__value.type'],
    model=model_name,
    view='events',
   filters={'events.event_date':'7 days'}
)
# result_format can also be sql
event_properties = sdk.run_inline_query(body=query, result_format='json',cache=True)


query = looker_sdk.models40.WriteQuery(
   fields=['events__user_properties.key','events__user_properties__value.type'],
    model=model_name,
    view='events',
   filters={'events.event_date':'7 days'}
)
# result_format can also be sql
user_properties = sdk.run_inline_query(body=query, result_format='json',cache=True)

In [ ]:
import ast
print("#### User Properties ####")
print("view: user_properties_generated {")
print("extension: required")
print()

for event in ast.literal_eval(user_properties)[1:]: #skip over the first one 
    key = event['events__user_properties.key']
    type = event['events__user_properties__value.type']
    
    
    print("dimension: user_properties."+key+" {")
    print("type: "+ type)
    print("sql:")
    if  type == "string":
        print("   (SELECT value.string_value")
    else:
        print("   (SELECT value.int_value")
    print("       FROM UNNEST(${user_properties})")
    print(   "       WHERE key = '"+key+"') ;;")
    print("  }")
    print()
print ("}")

In [ ]:
print("#### Event Properties ####")
print("view: events_generated {")
print("extension: required")
print()
for event in ast.literal_eval(event_properties)[1:]: #skip over the first one
    event_name = event['events.event_name']
    type = event['events__event_params__value.type']
    key = event['events__event_params.key']
  
    print("dimension: "+event_name+"."+key+" {")
    print("type: "+ type)
    print("sql: CASE WHEN ${event_name} = '"+event_name+"' THEN")
    if  type == "string":
        print("   (SELECT value.string_value")
    else:
        print("   (SELECT value.int_value")
    print("       FROM UNNEST(${event_params})")
    print(   "       WHERE key = '"+key+"')")
    print(   " END ;;")
    print("  }")
    print()
print("}")

  
